<img style="float: right;"  src="images/LogoP.jpg" width="200">

# SLab Demo : Firmware Test

This a **demo** Jupyter Notebook for the SLab projects

Version 1.0 (3/6/2018) License information is at the end of the document

---

The SLab user interface is based on **Python** commands that control the **Hardware Board** operation.

The board host a **SLab Firmware** that works as a server for commands sent, through serial connection, from the PC.

This document briefly explains the **firmware commands** the board can understand and show, for each command, at least one **Python** SLab module function that uses it.

This document can be used to check the firmware operation, as all commands defined in the SLab modules use the firmware commands described in this document.

The document verifies that the board can execute all firmware commands, but it does not verify that it works with any parameter combination. In the same way, the document is not designed to check the proper operation of the board hardware.

## Firmware Commands

All SLab commands rely on the functionalitios available on the Hardware Board thanks to its firmware. Those functionalities are associated to a set of commands. Currently, SLab boards could implement commands in several cathegories:

### Initialization and Configuration commands

* **F** : Get Firmware String
* **M** : Get Magic Code
* **I** : Get Board Information
* **L** : Get Pin List
* **E** : Soft Reset

### DC Analog Commands

* **A** : Read ADC channel
* **D** : Write DAC channel
* **N** : Number of reads in DC

### DC Digital I/O commands

* **H** : DIO Mode
* **J** : DIO Write
* **K** : DIO Read

### Transient commands

* **R** : Set sample period
* **S** : Set transient storage
* **Y** : Async Read
* **G** : Triggered Read
* **P** : Step Response
* **W** : Load wavetable
* **w** : Load secondary wavetable
* **V** : Wave response
* **v** : Dual wave response
* **Q** : Wave play
* **q** : Dual wave play

If a commands is not understood by the **Hardware Board** it will give a **Remote Error** like:

$\qquad$ **Remote Error : Bad command parameters**

The same error will be shown if you select a proper command with bad parameters.

This demo Notebook uses a minimum subset of the SLab commands that is enough to check all firmware functionalities.

The Notebook is not intended to learn about the SLab Commands. Use the reference or tutorial Notebooks for that.

## Module import

First we will import the main SLab module
This code sould give some messages including the SLab version

In [ ]:
# Import all slab modules
import slab

## Interactive plots

Executing the following code we can make our plots interactive. That way you can zoom and pan on the plot.

When you are done with the interaction, just hit the close interaction button.

In [ ]:
# Make plots interactive
slab.interactivePlots()
%matplotlib notebook

## Connection and board information

This code uses the **connect** command to connect to the board using **autodetect** of the COM port
You should see text information about the board and the COM port used

Afterwards we show the information about the board using the **printBoardInfo** command.

The code checks the firmware commands:

* **F** : Get Firmware String. It gives information about the Firmware version.
* **M** : Get Magic Code. Used to check that the response if from a SLab firmware.
* **I** : Get information about the board capabilities.
* **L** : Get Pin List from the board.

As the board capabilities are provided by the board itself, you con't need to configure anything on SLab if you change to another SLab compatible board.

In [ ]:
boardFolder = ''                                # Board folder (leave '' if you use only one board)
slab.setFilePrefix('../Files/')                 # Set File Prefix
slab.setCalPrefix('Calibrations/'+boardFolder)  # Set Calibration Prefix         
slab.connect()                                  # Connect to the board

In [ ]:
# Show board information
slab.printBoardInfo()

## Set DAC and read ADC

This code sets voltages on the DACs and read voltages on the ADCs

In general, ADCs are more reliable that DACs. If the node has an impedance below 100k Ohm, the ADC reading should be quite good even on uncalibrated SLab systems.

The DAC output voltage, however, depends on the load, so they are not as reliable as the ADCs. In general, it is a good idea to put an ADC at each DAC output to obtain the **real** voltage delivered by the DAC. 

The code sets the voltago of **DAC1** and **DAC2** and reads all ADC voltages.

You can connect some ADCs to the DAC outputs or to Vdd or GND to see if the all give the proper readings.

The code checks the firmware commands:

* **A** : Read and ADC channel
* **D** : Write a DAC channel
* **N** : Set number of reads in DC

In [ ]:
# Set DACs
slab.setVoltage(1,2.0)
slab.setVoltage(2,1.0)

# Get all voltages
slab.dcPrint()

To improve the readings, you can instruct SLab to perform several readings and give the average of them.

In [ ]:
# Average 10000 readings on each ADC
last = slab.setDCreadings(10000);

# Get all voltages
slab.dcPrint()

# Return to previous operation
slab.setDCreadings(last);

## Board Memory Buffer

Most of the RAM memory in the **Hardware Board** is used for an **unified memory buffer (UMB)**. The UMB is used for storing:

* Transient measurements obtained on ADCs
* Wavetables for operating the DACs

Wavetables are always on the bottom region of the UMB whereas measurements are in the top region.

The board memory is measured on samples. One sample is equivalent to an unsigned 16 bit number. All measurements are performed by the **board firmware** in ratiometric units (between 0 and 1). The conversion to voltages is made in Python code. 

The codification of the ratiometric values are made in 16 bits and are independent on the number of bits of the ADC and DAC converters. That makes the Python interpetation of the ratiometric values independent on the kind of converters the board uses.

* Ratiometric 0 that corresponds to **GND**: $0$
* Ratiometric 1 that corresponds to **Vref**: $2^{16}-1$

The **UMB** is cleared when you connect with the board and when you perform a **Soft Reset**.

## Transient Async

The transient async command measures one to four ADC channels during a given time.

The **setSampleTime** command determines the time between two consecutive measurements.

The **setTransientStorage** command, or its shorter alias **tranStore** command, reserves memory on the **UMB** for an given number of samples on a given number of ADC lines.

The following code reads four ADCs for a total of 100 samples of each ADC waiting $15\mu s$ between readings. After the measuremenmt, the results are shown on a plot.

The code tests the following firmware commands:

* **R** : Set sample period
* **S** : Set transient storage
* **Y** : Async Read

In [ ]:
# Set sample time
slab.setSampleTime(0.000015)

# Set transient storage of 100 samples of all 4 ADC channels
slab.tranStore(100,4)

# Perform the measurement and show as a plot
slab.tranAsyncPlot()

## Triggered Read

The triggered read command is similar to the previous **Transient Async** command. The only difference is that it waits for a given condition in one analog channel to sync the capture.

The triggering condition will be at time zero, just in the center of the capture region.

The code tests the following firmware commands:

* **R** : Set sample period
* **S** : Set transient storage
* **G** : Triggered Read

In [ ]:
# Set sample time
slab.setSampleTime(0.000015)
slab.setSampleTime(0.001)

# Set transient storage of 100 samples of all 4 ADC channels
slab.tranStore(100,4)

# Perform the measurement and show as a plot
slab.tranTriggeredPlot(1.0,slab.tmodeFall)

## Step response

The step response command give the response of a circuit against a step change on **DAC 1**

1/5 of the samples will be captured before the step and 4/5 afterwards

Time zero will be defined as the step position.

The code tests the following firmware commands:

* **R** : Set sample period
* **S** : Set transient storage
* **P** : Step Response

In [ ]:
# Set sample time
slab.setSampleTime(0.000015)

# Set transient storage of 100 samples of all 4 ADC channels
slab.tranStore(100,4)

# Perform the measurement and show as a plot
slab.stepPlot(1.0,2.0)

## Wave Response Commands

The wave commands obtain the response of a circuit agains a user defined waveform.

Before generating a waveform on the **hardware board** it needs to be previously loaded. The board can store up to two different waveforms at the same time. One principal one for **DAC1** and another secondary one for **DAC2**.

To ease the memory management, the **UMB** is cleared every time you upload the primary waveform. That means that, if you change the primary waveform, the secondary waveform is cleared.

As the **UMB** is unified for waveforms and measurements, the upload of waveforms reduce the space available for measurements.

The wave response commands generate a wave, and obtain a response of the circuit to this wave. There are three kinds of wave response commands available:

The **waveResponse** commands generate a wave on **DAC1** and obtain a response on one or several ADCs.

The **dualWaveResponse** commands generate waves both on **DAC1** and **DAC2** and obtain a response on one or several ADCs.

The **singleWaveResponse** command generate a wave on **DAC1** and obtain a response on one selected ADC channel. As only one channel is read on this command, it is usually available on all ADC channels, not only on the ones available to be read a the same time. Moreover, as we only read one channel, this is usually wave response command that ca use lower sample times.

The code tests the following firmware commands:

* **R** : Set sample period
* **S** : Set transient storage
* **W** : Load wavetable
* **w** : Load secondary wavetable
* **V** : Wave response
* **v** : Dual wave response
* **x** : Single Wave response


In [ ]:
# Wave Response Example

# Load a 100 point sine wave that goes from 1V to 2V
slab.waveSine(1,2,100)

# Set the wave frequency to 100Hz
slab.setWaveFrequency(100)

# Set storage for all four ADCs for two waves (200 samples)
slab.tranStore(200,4)

# Generate the wave and see the circuit response
slab.wavePlot()

In [ ]:
# Dual Wave Response Example

# Load a DAC1 100 point sine wave that goes from 1V to 2V
slab.waveSine(1,2,100)

# Set the wave frequency to 100Hz
slab.setWaveFrequency(100)

# Load a DAC2 50 sample secondary sine waveform between 0.5V and 2.5V
slab.waveSine(0.5,2.5,50,second=True)

# Set storage for all four ADCs for two waves of the primary wave (200 samples)
slab.tranStore(200,4)

# Generate the waves on both DACs and see the circuit response
slab.wavePlot(dual=True)

In [ ]:
# Single Wave Response Example

# Load a DAC1 100 point sine wave that goes from 1V to 2V
slab.waveSine(1,2,100)

# Set the wave frequency to 100Hz
slab.setWaveFrequency(100)

# Set storage for all one ADC for two waves of the primary wave (200 samples)
slab.tranStore(200,1)

# Generate the waves on both DACs and see the circuit response
slab.singleWavePlot(8)

## Wave Play

The wave play command generates a wave but don't capture any data

Waves can be generated on **DAC1** or both on **DAC1** and **DAC2**

As no measurements are stored, the wave can be played forever by setting a zero time. In that case, you need to use the **halt** button to stop the play. 

The code tests the following firmware commands:

* **R** : Set sample period
* **S** : Set transient storage
* **W** : Load wavetable
* **w** : Load secondary wavetable
* **Q** : Wave play
* **q** : Dual wave play

In [ ]:
# Wave Play Example

# Load a 100 point sine wave that goes from 1V to 2V
slab.waveSine(1,2,100)

# Set the wave frequency to 100Hz
slab.setWaveFrequency(100)

# Generate 10 waveforms
slab.wavePlay(10)

In [ ]:
# Dual Wave Play Example

# Load a DAC1 100 point sine wave that goes from 1V to 2V
slab.waveSine(1,2,100)

# Set the wave frequency to 100Hz
slab.setWaveFrequency(100)

# Load a DAC2 50 sample secondary sine waveform between 0.5V and 2.5V
slab.waveSine(0.5,2.5,50,second=True)

# Generate 10 waveforms of the primary wave on DAC1
# Generate also the DAC2 wave during the DAC1 generation
slab.wavePlay(10,dual=True)

In [ ]:
# Infinite Wave Play Example

# Load a 100 point sine wave that goes from 1V to 2V
slab.waveSine(1,2,100)

# Set the wave frequency to 100Hz
slab.setWaveFrequency(100)

# Generate waveforms forever
# Stop the play pushing the hardware button
slab.wavePlay(0)

## DC Digital I/O

The digital I/O lines (DIO) can be configured in different input and output modes

The DC digital I/O commands set the DIO configuration, set the state of the outputs and read the state of the inputs

The code tests the following firmware commands:

* **H** : DIO Mode
* **J** : DIO Write
* **K** : DIO Read

In [ ]:
# Configure DIO 1 as output 
slab.dioMode(1,slab.mOutput)

# Configure DIO 2 as input
slab.dioMode(2,slab.mInput)

# Read DIO 2
print(slab.dioRead(2))

# Set DIO to High
slab.dioWrite(1,1)

# Read DIO 2
print(slab.dioRead(2))

# Wait 0.2 seconds
slab.wait(0.2)

# Set DIO to Low
slab.dioWrite(1,0)

## Soft Reset

The Soft Reset command enables the user to set the SLab board on known state.

As several of the firmware commands change the board state, it is sometimes usefull to set the board on a known state without needing to perform a **Hard Reset**.

The code tests the following firmware command:

* **E** : Soft Reset

In [ ]:
slab.softReset()

## Disconnect from the board

The following command disconnects from the board

It just closes the serial port, so no firmware command is issued

In [ ]:
# Disconnect from the board
slab.disconnect()

## Document license

Copyright  ©  Vicente Jiménez (2018)  
This work is licensed under a Creative Common Attribution-ShareAlike 4.0 International license.  
This license is available at http://creativecommons.org/licenses/by-sa/4.0/

<img  src="images/cc_sa.png" width="200">